In [1]:
import pandas as pd

CSV_PATH = 'C:/Users/madad/Documents/features-train.csv'
new_features_df = pd.read_csv(CSV_PATH, header=0, usecols=['blurrness','image_size','whiteness', 'average_pixel_width','dullness', 'PetID'])

new_features_df.head(5)

,PetID,dullness,whiteness,average_pixel_width,image_size,blurrness
0,0008c5398-1.jpg,0.000,0.000,7.153320,23966,108542.25200
1,0008c5398-2.jpg,87.820,0.000,9.179688,19326,71455.78827
2,0008c5398-3.jpg,4.885,0.000,7.446289,19778,65935.54666
3,0008c5398-4.jpg,40.885,1.725,10.205078,23557,64761.50388
4,0008c5398-5.jpg,0.000,0.000,8.129883,20335,63494.16191


In [2]:
CSV_PATH = 'C:/Users/madad/Documents/all_train_data_V6_pure_breed.csv'
df = pd.read_csv(CSV_PATH, header=0)

df.head(5)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,NMF_img_tags_1,NMF_img_tags_2,NMF_img_tags_3,NMF_img_tags_4,RescuerID_COUNT,main_breed_Type,main_breed_BreedName,second_breed_Type,second_breed_BreedName,PureBreed
0,2,Ollie,6,265,0,1,4,7,0,2,...,0.118481,0.0,0.000000,0.000000,2,2.0,0,NaN,-1,0.0
1,1,Cutie Pie,2,307,0,2,1,2,7,2,...,0.000000,0.0,0.038738,0.000000,93,1.0,1,NaN,-1,0.0
2,1,Minnie,2,307,0,2,1,0,0,2,...,0.000800,0.0,0.011280,0.005097,8,1.0,1,NaN,-1,0.0
3,2,Spot,2,241,241,1,1,4,6,2,...,0.116961,0.0,0.000000,0.000000,2,2.0,2,2.0,0,1.0
4,1,Pup,1,307,0,1,2,5,0,2,...,0.000000,0.0,0.083036,0.000000,2,1.0,1,NaN,-1,0.0


In [3]:
new_features_df = new_features_df.sort_values(by=['PetID'])
new_features_df['PetID'].head(10)

0    0008c5398-1.jpg
1    0008c5398-2.jpg
2    0008c5398-3.jpg
3    0008c5398-4.jpg
4    0008c5398-5.jpg
5    0008c5398-6.jpg
6    000a290e4-1.jpg
7    000a290e4-2.jpg
8    000fb9572-1.jpg
9    000fb9572-2.jpg
Name: PetID, dtype: object

In [4]:
df = df.sort_values(by=['PetID'])
df['PetID'].head(5)

0    0008c5398
1    000a290e4
2    000fb9572
3    0011d7c25
4    00156db4a
Name: PetID, dtype: object

In [5]:
def make_row(pet_id, sums, size):
    return extra_df.append({'PetID': pet_id, 'dullness': sums[0]/size, 'blurrness': sums[1]/size,
                           'image_size': sums[2]/size, 'average_pixel_width': sums[3]/size,
                           'whiteness': sums[4]/size}, ignore_index=True)

In [6]:
def init_sums(curr_pet):
    dull_sum = curr_pet['dullness']
    blurr_sum = curr_pet['blurrness']
    avg_pixel_width_sum = curr_pet['average_pixel_width']
    whiteness_sum = curr_pet['whiteness']
    size_sum = curr_pet['image_size']
    return dull_sum, blurr_sum, size_sum, avg_pixel_width_sum, whiteness_sum

In [7]:
def feature_sums(curr_pet, sums):
    dull_sum = sums[0] + curr_pet['dullness']
    blurr_sum = sums[1] + curr_pet['blurrness']
    avg_pixel_width_sum = sums[2] + curr_pet['image_size']
    whiteness_sum = sums[3] + curr_pet['average_pixel_width']
    size_sum = sums[4] + curr_pet['whiteness']
    return dull_sum, blurr_sum, size_sum, avg_pixel_width_sum, whiteness_sum

In [8]:
# merge new features data frame for pet
extra_df = pd.DataFrame(columns = ['blurrness','image_size','whiteness', 'average_pixel_width','dullness']) 

prev_pet = new_features_df.iloc[0]
sums = init_sums(prev_pet)

size = 1

for i in range (1, len(new_features_df)):
    curr_pet = new_features_df.iloc[i]
    curr_pet_id = curr_pet['PetID'].split('-')[0]
    prev_pet_id = prev_pet['PetID'].split('-')[0]
    
    if curr_pet_id == prev_pet_id:
        sums = feature_sums(curr_pet, sums)
        size = size + 1
        
        if i == len(new_features_df['PetID']) - 1:
            extra_df = make_row(prev_pet_id, sums, size)
    else:
        extra_df = make_row(prev_pet_id, sums, size)
        # reinitialize sums for next pet
        sums = init_sums(curr_pet)
        size = 1

    prev_pet = curr_pet

In [9]:
print(len(extra_df))
print(len(df))
print(len(df)-len(extra_df),'pets do not have images.')

14651
14993
342 pets do not have images.


In [10]:
extra_df['PetID'].head(5)

0    0008c5398
1    000a290e4
2    000fb9572
3    0011d7c25
4    00156db4a
Name: PetID, dtype: object

In [11]:
# there are pets that do not have attached images
# more photos per pet
assert(len(df) >= len(extra_df))

In [13]:
def make_row_default_on_pos(pet_id, pos, df):
    row = pd.DataFrame({'PetID': pet_id, 'dullness': 101, 'blurrness': 100000,
                           'image_size': 0, 'average_pixel_width': -1,
                           'whiteness': 101}, index=[pos])
    df = df.append(row, ignore_index=False)
    df = df.sort_index().reset_index(drop=True)
    return df

In [14]:
# parse df (all pet ids)
# if there is no image for that id, then add defaults for the new features in extra_df

count = 0

for i in range(0, len(df)):
    try:
        pet_id_df = df.iloc[i]['PetID']
        pet_id_extra_df = extra_df.iloc[i]['PetID']
    except:
        # the last element does not have photos
        print(i)
        print(len(extra_df))
        print(len(df))
    
    if pet_id_df != pet_id_extra_df:
        count = count + 1
        # if the df does not correspond to extra_df, then insert default row on position i 
        extra_df = make_row_default_on_pos(pet_id_df, i-1, extra_df)

print('Added default values for', count, 'pets')

c:\users\madad\venv\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


14992
14993
Added default values for 342 pets


In [15]:
extra_df.head()

assert(len(df) == len(extra_df))

In [16]:
extra_df = extra_df.drop('PetID', 1)

In [17]:
df = df.join(extra_df)

In [18]:
df.to_csv('all_train_data_V7.csv', encoding='utf-8', index=False)